In [1]:
# Install required Python libraries
!pip install rpy2 matplotlib pandas

# Load rpy2 to run R code in the notebook
%load_ext rpy2.ipython

In [11]:
### Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
%%R

# Load required libraries
library(data.table)
library(ggplot2)
library(dplyr)

# Load pre-cleaned NASS 2020 data (assume available in Colab environment)
NASS_2020_all <- fread("/content/drive/My Drive/test.csv")

# Verify data structure
str(NASS_2020_all)

Classes ‘data.table’ and 'data.frame':	0 obs. of  0 variables
 - attr(*, ".internal.selfref")=<externalptr> 


In addition: Warning messages:
1: In (if (.Platform$OS.type == "unix") system else shell)(paste0("(",  :
  error in running command
2: In fread("/content/drive/My Drive/test.csv") :
  File '/tmp/Rtmp34bHuk/file54c668541c0' has size 0. Returning a NULL data.table.


In [13]:
#load data

%%R
library(data.table)

# 1. Check if file exists:
file_path <- "/content/drive/My Drive/test.csv"
if (file.exists(file_path)) {
  print("File found!")
} else {
  print(paste("File not found at:", file_path))
  # If not found, double-check the path or upload the file to the correct location
  stop("File not found. Exiting.")
}

# 2. Attempt to load using fread:
NASS_2020_all <- fread(file_path)

# 3. Verify data structure:
str(NASS_2020_all)

[1] "File found!"
Classes ‘data.table’ and 'data.frame':	371290 obs. of  17 variables:
 $ file   : int  15 15 15 15 15 15 15 15 15 15 ...
 $ trans  : int  735496 735496 735496 735496 735496 735496 735496 735496 735496 735496 ...
 $ date   : chr  "9/21/2015" "9/21/2015" "9/21/2015" "9/21/2015" ...
 $ time   : chr  "17:00:05" "17:00:10" "17:00:15" "17:00:20" ...
 $ dtnum  : num  42269 42269 42269 42269 42269 ...
 $ dsi    : int  17 17 17 17 17 17 17 17 17 17 ...
 $ rdn    : int  -45 -45 -45 -45 -45 -45 -45 -45 -45 -45 ...
 $ hfhs   : int  -11 -11 -11 -11 -11 -11 -11 -11 -11 -11 ...
 $ term   : int  -79 -79 -79 -79 -79 -79 -79 -79 -79 -79 ...
 $ deli   : int  45 45 45 45 45 45 45 45 45 45 ...
 $ abstart: int  -20 -20 -20 -20 -20 -20 -20 -20 -20 -20 ...
 $ abend  : int  -64 -64 -64 -64 -64 -64 -64 -64 -64 -64 ...
 $ dn     : chr  "Day" "Day" "Day" "Day" ...
 $ exp    : chr  "RDN" "RDN" "RDN" "RDN" ...
 $ hr     : int  95 107 105 101 111 107 97 111 104 99 ...
 $ sys    : num  151 147 147 14

In [ ]:
%%R
### Age and Sociodemographic Diversity Analysis

# Create a dummy variable for White individuals
NASS_2020_all[, WHITE := ifelse(RACE == 1, 1, 0)]

# Calculate unadjusted proportion of White individuals
unadjusted_proportion_white <- mean(NASS_2020_all$WHITE)
print(paste("Unadjusted proportion of WHITE:", unadjusted_proportion_white))

# Perform weighted analysis using survey package
design <- svydesign(ids = ~KEY_NASS, weights = ~DISCWT, data = NASS_2020_all)
weighted_proportion_white <- svymean(~WHITE, design)
print(paste("Weighted proportion of WHITE:", coef(weighted_proportion_white)))

# Save results to a CSV file
write.csv(data.frame(Unadjusted = unadjusted_proportion_white, Weighted = coef(weighted_proportion_white)), "/content/White_Proportion_Analysis.csv", row.names = FALSE)

In [ ]:
%%R
### Generate Overview Graphs

# Define labels for factor variables
teach_labels <- c("0" = "Non-Teaching", "1" = "Teaching")
location_labels <- c("0" = "Rural", "1" = "Urban")
bed_labels <- c("1" = "0-99", "2" = "100-299", "3" = "300+")
region_labels <- c("1" = "Northeast", "2" = "Midwest", "3" = "South", "4" = "West")
race_labels <- c("1" = "White", "2" = "Black", "3" = "Hispanic", "4" = "Asian/Pacific", "5" = "Native", "6" = "Other")

# Define a consistent theme for all plots
custom_theme <- theme_minimal() +
  theme(text = element_text(size = 12), axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "bottom", legend.title = element_blank())

# FIGURE 1: Hospitals by Region and Bed Size
fig1 <- ggplot(NASS_2020_all, aes(x = HOSP_REGION, fill = HOSP_BEDSIZE_CAT)) +
  geom_bar() +
  custom_theme +
  xlab("US Region") +
  ggtitle("Hospitals within NASS 2020 Dataset", subtitle = "Segmented by Bed Size Category")

# Save the plot
ggsave("/content/Hospitals_by_Region_and_Bed_Size.png", plot = fig1)

# FIGURE 2: Age Distribution by Race
fig2 <- ggplot(NASS_2020_all, aes(x = AGE, fill = RACE)) +
  geom_density(alpha = 0.5) +
  custom_theme +
  xlab("Age") +
  ggtitle("Age Distribution by Race")

# Save the plot
ggsave("/content/Age_Distribution_by_Race.png", plot = fig2)

In [ ]:
%%R
### Save Outputs

# Save cleaned data and graphs
fwrite(NASS_2020_all, "/content/NASS_2020_all_cleaned.csv")
print("All outputs saved to Colab environment.")

In [ ]:
%%R
### Poster Space Setup

# Combine all plots into a single PDF for presentation
pdf("/content/NASS_Analysis_Poster.pdf", width = 12, height = 8)

# Add Figure 1
print(fig1)

# Add Figure 2
print(fig2)

# Close the PDF
dev.off()

# Save cleaned data for poster reference
fwrite(NASS_2020_all, "/content/NASS_2020_all_cleaned.csv")
print("Poster PDF and cleaned data saved to Colab environment.")